In [13]:
with open("input16.txt") as f:
    inp = [x[:-1] for x in f.readlines()][0]
inp

'60556F980272DCE609BC01300042622C428BC200DC128C50FCC0159E9DB9AEA86003430BE5EFA8DB0AC401A4CA4E8A3400E6CFF7518F51A554100180956198529B6A700965634F96C0B99DCF4A13DF6D200DCE801A497FF5BE5FFD6B99DE2B11250034C00F5003900B1270024009D610031400E70020C0093002980652700298051310030C00F50028802B2200809C00F999EF39C79C8800849D398CE4027CCECBDA25A00D4040198D31920C8002170DA37C660009B26EFCA204FDF10E7A85E402304E0E60066A200F4638311C440198A11B635180233023A0094C6186630C44017E500345310FF0A65B0273982C929EEC0000264180390661FC403006E2EC1D86A600F43285504CC02A9D64931293779335983D300568035200042A29C55886200FC6A8B31CE647880323E0068E6E175E9B85D72525B743005646DA57C007CE6634C354CC698689BDBF1005F7231A0FE002F91067EF2E40167B17B503E666693FD9848803106252DFAD40E63D42020041648F24460400D8ECE007CBF26F92B0949B275C9402794338B329F88DC97D608028D9982BF802327D4A9FC10B803F33BD804E7B5DDAA4356014A646D1079E8467EF702A573FAF335EB74906CF5F2ACA00B43E8A460086002A3277BA74911C9531F613009A5CCE7D8248065000402B92D47F14B97C723B953C7B22392788A7CD62C1EC0

# Both parts

In [39]:
# Auxiliary function
def prod(l):
    res = 1
    for x in l:
        res = res*x
    return res

# Implementing a tree-like class of packets
class Packet():
    
    def __init__(self, bits):
        self.bits = bits
        self.version = int(bits[:3],2)
        self.typeid = int(bits[3:6],2)
        if self.typeid == 4:
            self.num = self.parse_value(bits[6:])[0]
            self.end = self.parse_value(bits[6:])[1]
            self.version_sum = self.version
            self.result = self.num
        else: 
            self.packets = self.parse_packets(bits[6:])[0]
            self.end = self.parse_packets(bits[6:])[1]
            self.version_sum = self.version + sum([x.version_sum for x in self.packets])
            if self.typeid == 0:
                self.result = sum([x.result for x in self.packets])
            elif self.typeid == 1:
                self.result =  prod([x.result for x in self.packets])
            elif self.typeid == 2:
                self.result =  min([x.result for x in self.packets])
            elif self.typeid == 3:
                self.result =  max([x.result for x in self.packets])
            elif self.typeid == 5:
                self.result =  1 if self.packets[0].result > self.packets[1].result else 0
            elif self.typeid == 6:
                self.result =  1 if self.packets[0].result < self.packets[1].result else 0
            elif self.typeid == 7:
                self.result =  1 if self.packets[0].result == self.packets[1].result else 0
        
            
    def parse_value(self, x):
        """Returns both value and ending index"""
        num = ""
        i=0
        while True:
            num += x[i+1:i+5]
            i += 5
            if x[i-5] == '0':
                break
        return int(num, 2), 6+i
    
    def parse_packets(self, x):
        if x[0] == '0':
            values = []
            length = int(x[1:16], 2)
            i = 16
            while i < 16 + length:
                packet = Packet(x[i:])
                i += packet.end
                values.append(packet)
            return values, 6+i
                
        if x[0] == '1':
            values = []
            npackets = int(x[1:12], 2)
            i = 12
            for _ in range(npackets):
                packet = Packet(x[i:])
                i += packet.end
                values.append(packet)
            return values, 6+i             

In [40]:
def hex_to_bin(inp):
    bits = []
    for i in range(len(inp)):
        bit = str(bin(int(inp[i],16)))[2:]
        while len(bit)<4:
            bit = '0'+bit
        bits.append(bit)
    return ''.join(bits)

bits = hex_to_bin(inp)
packet = Packet(bits)

In [41]:
# Part 1
packet.version_sum
# Part 2
packet.result

68703010504